# December 12, 2022
https://adventofcode.com/2022/day/12

Wrong answers  
453 (too high?)

The very end of the path is wrong for some reason... why does it go up and around instead of under the E?

In [1]:
from queue import PriorityQueue

In [2]:
test = ["Sabqponm\n"
, "abcryxxl\n"
, "accszExk\n"
, "acctuvwj\n"
, "abdefghi\n"]

test

['Sabqponm\n', 'abcryxxl\n', 'accszExk\n', 'acctuvwj\n', 'abdefghi\n']

In [3]:
fn = "data/12.txt"
with open(fn, "r") as file:
    puz = file.readlines()

In [112]:
class Map:
    def __init__( self, text ):
        self.map = [ [c for c in line.strip("\n")] for line in text ]
        self.h = len(self.map)
        self.w = len(self.map[0])
        self.start = None
        self.goal = None

        # Look for start and end
        # Might consider updating heights for these places
        for y in range(self.h):
            for x in range(self.w):
                if self.map[y][x] == "S":
                    self.start = self.get_id( [y,x] )
                    if self.goal is not None:
                        break
                elif self.map[y][x] == "E":
                    self.goal = self.get_id( [y,x] )
                    if self.goal is not None:
                        break
            if self.goal is not None and self.start is not None:
                break

    def height(self, yx):
        '''height at yx coord pair'''
        if self.map[yx[0]][yx[1]] == "S":
            ht = "a"
        elif self.map[yx[0]][yx[1]] == "E":
            ht = "z"
        else:
            ht = self.map[yx[0]][yx[1]]

        return ord(ht) - 97 # ord("a")

    @classmethod
    def get_coords( cls, id ):
        '''get coords given an id'''
        return [ int(coord) for coord in id.split(".") ]
    @classmethod
    def get_id( cls, yx ):
        '''get id from coords'''
        return ".".join( [str(coord) for coord in yx] )


    @classmethod
    def est_distance( cls, id0, id1 ):
        # estimated distance from (y0,x0) to (y1,x1)
        # this ignores path restrictions
        yx0 = cls.get_coords(id0)
        yx1 = cls.get_coords(id1)
        #return 0
        return abs(yx1[0] - yx0[0]) + abs(yx1[1] - yx0[0])

    def edge_weight( self, id0, id1 ):
        '''edge weight from id0 to id1'''
        # this is directional!
        yx0 = self.get_coords(id0)
        yx1 = self.get_coords(id1)
        
        if self.height(yx0) >= self.height(yx1) - 1:
            return 1
        else:
            return None
        

    def get_neighbors( self, id ):
        '''get neighbors given the id'''
        # get neighbors, checking for edge conditions
        yx = self.get_coords(id)
        nbrs = []
        if yx[0] > 0:
            maybe = self.get_id( [yx[0]-1, yx[1]] )
            if self.edge_weight(id, maybe) is not None:
                nbrs.append(maybe)
        if yx[1] > 0:
            maybe = self.get_id( [yx[0], yx[1]-1 ] )
            if self.edge_weight(id, maybe) is not None:
                nbrs.append(maybe)
        if yx[0] < self.h - 1:
            maybe = self.get_id( [yx[0]+1, yx[1]] )
            if self.edge_weight(id, maybe) is not None:
                nbrs.append( maybe )
        if yx[1] < self.w - 1:
            maybe = self.get_id( [yx[0], yx[1]+1] )
            if self.edge_weight(id, maybe) is not None:
                nbrs.append( maybe )
        return nbrs

    # for Part 2
    def all_starts( self ):
        starts = []
        for y, line in enumerate(self.map):
            for x, char in enumerate(line):
                if char == "S" or char == "a":
                    starts.append( self.get_id( [y,x] ) )
        return starts

    

                    

In [102]:
def astar( map, starts, goal ):
    # if restarts is True, check if each tile can count as a starting square
    frontier = PriorityQueue() # est distance, object, distance from start
    dist_so_far = dict() # distance from start to key
    parents = dict() # preceding tile to key on the path

    # add all possible start tiles to frontier
    for s in starts:
        frontier.put( (0, s) ) # typically priority is est dist to goal, but not for start
        dist_so_far[s] = 0
        parents[s] = None

    explored = list() # object, parent, distance, path

    while not frontier.empty():
        cur = frontier.get()[1]
        dist_to_cur = dist_so_far[cur]

        if cur == goal:
            break
        
        for nbr in map.get_neighbors( cur ):
            edge_weight = map.edge_weight( cur, nbr )
            path_dist = dist_so_far[cur] + edge_weight

            if nbr not in dist_so_far or path_dist < dist_so_far[nbr]:
            # Either node is new or we found a shortcut
                frontier.put( (path_dist + map.est_distance(nbr, goal), nbr) )
                dist_so_far[nbr] = path_dist
                parents[nbr] = cur

    return dist_so_far[goal], parents

def construct_path( map, parents ):
    final_path = [ map.goal ]
    while parents[ final_path[0] ] is not None:
        final_path.insert( 0, parents[final_path[0]] )
    return final_path

In [103]:
def print_path( m, path ):
    char_dict = { path[-1]: "E" }
    cur = path[0]
    y0,x0 = [int(c) for c in path[0].split(".")]
    for step in path[1:]:

        y1,x1 = [int(c) for c in step.split(".")]
        if y1-y0 == 1:
            char = "v"
        elif y0-y1 == 1:
            char = "^"
        elif x1-x0 == 1:
            char = ">"
        elif x0-x1 == 1:
            char = "<"
        else:
            print(cur, step, x0, y0, x1, y1)
            raise Exception("No Direction")

        char_dict[cur] = char
        cur = step
        x0=x1
        y0=y1


    for y in range(m.h):
        for x in range(m.w):
            id = f"{y}.{x}"
            if id in char_dict:
                char = char_dict[id]
            else:
                char = "."
            print(char, end="")
        print("\n", end="")

### Part 1

In [113]:
m = Map(test)

In [121]:
dist, parents = astar(m, [m.start], m.goal)
print_path(m, construct_path(m, parents))

Shortest Path: 31
v..v<<<<
>v.vv<<^
.>vv>E^^
..v>>>^^
..>>>>>^


In [122]:
puz_map = Map(puz)

In [123]:
dist, parents, = astar(puz_map, [puz_map.start], puz_map.goal)
final_path = construct_path(puz_map, parents)
print("Shortest Path:", dist)
print_path(puz_map, final_path)

Shortest Path: 447
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
.....................

...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
...............................................................................................................................................................
........................................

### Part 2

In [124]:
dist, parents = astar( m, m.all_starts(), m.goal )
print(dist)

29


In [125]:
dist, parents = astar( puz_map, puz_map.all_starts(), puz_map.goal )
dist

446